# IBM Data Science Capstone Project - The Battle of Neighborhoods

# Looking for a property in Sydney

## Backgroud

A client came from Hong Kong is looking for an Sydney comforable home which is close to a train station within 20km from the Sydney central business district (CBD). Before deciding which location to look at, the clients wants to have a sense of community and neighbourhood of each train station within 20km from the Sydney CBD. As a data analyst in the property company, I am asked to provide a neighbourhood analysis of all possible train stations by using any online resources.

## Data

To solve the business problem, we plan to use the Foursquare API to obtain a list of train stations within 20km radius from the Sydney CBD. We will then explore and analyse each train station by using the explore function to get the most common venue categories, and use this feature to group the train station into clusters. We will use the k-means clustering algorithm and examine the charateristics of each clusters. We will have a glance over the top 5 most common venues of each train staions. We will then use the folium library to visualize the train stations in the Sydney city centre map and their emerging clusters. Finally, We will examine and provide a brief summary of each cluster.

## 1. Create a list of train stations within 20km radius of Sydney CBD

### 1.1 Download and import all the dependencies that we will need

In [1]:
import numpy as np # library to handle data in a vectorized mannerD
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 1.2 Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'T5F4IPH15CZ2MG15H2EDIVZIYZ2SD1YQESYECTRFARDYARPQ' # your Foursquare ID
CLIENT_SECRET = 'MEZMXAASYWY5YUDDL53ZMS4N5MMR3BEMUE3ARZKZDO3JEOCJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T5F4IPH15CZ2MG15H2EDIVZIYZ2SD1YQESYECTRFARDYARPQ
CLIENT_SECRET:MEZMXAASYWY5YUDDL53ZMS4N5MMR3BEMUE3ARZKZDO3JEOCJ


### 1.3 Get the latitude and longitude coordinates for Sydney CBD

In [3]:
syd_latitude = -33.8688
syd_longitude = 151.2093

### 1.4 Get all the train stations within 20km from Sydney CBD

In [4]:
radius=20000
LIMIT=500
train_categoryId='4bf58dd8d48988d129951735'

url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET, 
    VERSION, 
    syd_latitude, 
    syd_longitude,
    train_categoryId,
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?&client_id=T5F4IPH15CZ2MG15H2EDIVZIYZ2SD1YQESYECTRFARDYARPQ&client_secret=MEZMXAASYWY5YUDDL53ZMS4N5MMR3BEMUE3ARZKZDO3JEOCJ&v=20180605&ll=-33.8688,151.2093&categoryId=4bf58dd8d48988d129951735&radius=20000&limit=500'

In [5]:
#create the get request URL
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d47d67b075a42002f564f6a'},
 'response': {'venues': [{'id': '4b058770f964a520539322e3',
    'name': 'Wynyard Station',
    'location': {'address': 'George St.',
     'crossStreet': 'Underground',
     'lat': -33.865676078654374,
     'lng': 151.20616275025094,
     'labeledLatLngs': [{'label': 'display',
       'lat': -33.865676078654374,
       'lng': 151.20616275025094}],
     'distance': 452,
     'postalCode': '2000',
     'cc': 'AU',
     'city': 'Sydney',
     'state': 'NSW',
     'country': 'Australia',
     'formattedAddress': ['George St. (Underground)',
      'Sydney NSW 2000',
      'Australia']},
    'categories': [{'id': '4bf58dd8d48988d129951735',
      'name': 'Train Station',
      'pluralName': 'Train Stations',
      'shortName': 'Train Station',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/trainstation_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1564989051',
    'hasPerk': 

In [6]:
#Structure the json data into panda dataframe
venues = results['response']['venues']
#flatten JSON   
nearby_venues = json_normalize(venues)
nearby_venues.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,4b058770f964a520539322e3,George St.,AU,Sydney,Australia,Underground,452,"[George St. (Underground), Sydney NSW 2000, Au...","[{'label': 'display', 'lat': -33.8656760786543...",-33.865676,151.206163,NaN,2000,NSW,Wynyard Station,v-1564989051,NaN
1,"[{'id': '4f4531504b9074f6e4fb0102', 'name': 'P...",False,503cbdfe526201b155bb7f9a,Railway Pde.,AU,Lidcombe,Australia,NaN,15159,"[Railway Pde., Lidcombe NSW 2141, Australia]","[{'label': 'display', 'lat': -33.8637884995757...",-33.863788,151.045402,NaN,2141,NSW,Sprint Platform,v-1564989051,NaN
2,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,4b0b7143f964a520873123e3,Elizabeth St.,AU,Sydney,Australia,Liverpool St.,880,"[Elizabeth St. (Liverpool St.), Sydney NSW 200...","[{'label': 'display', 'lat': -33.8766996640586...",-33.876700,151.209732,NaN,2000,NSW,Museum Station (Concourse),v-1564989051,NaN
3,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,4b061fe1f964a520efe822e3,Beecroft Rd.,AU,Epping,Australia,NaN,15848,"[Beecroft Rd., Epping NSW 2027, Australia]","[{'label': 'display', 'lat': -33.7732193222571...",-33.773219,151.082284,NaN,2027,NSW,Epping Station (Concourse),v-1564989051,NaN
4,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,55adf7ff498e688326dce3ed,George St.,AU,Sydney,Australia,Underground at Druitt & Bathurst Sts,587,[George St. (Underground at Druitt & Bathurst ...,"[{'label': 'display', 'lat': -33.8736438265870...",-33.873644,151.206781,NaN,2000,NSW,Town Hall Station,v-1564989051,NaN


In [7]:
# filter columns, only keep the station name and it's latitude and longitude
filtered_columns = ['name', 'location.lat', 'location.lng']
station_list = nearby_venues[filtered_columns]
station_list.head()

,name,location.lat,location.lng
0,Wynyard Station,-33.865676,151.206163
1,Sprint Platform,-33.863788,151.045402
2,Museum Station (Concourse),-33.876700,151.209732
3,Epping Station (Concourse),-33.773219,151.082284
4,Town Hall Station,-33.873644,151.206781


In [8]:
#clean the station list

station_list = station_list[~station_list['name'].str.contains('Concourse')]
station_list = station_list[~station_list['name'].str.contains('Platform')]
station_list = station_list[~station_list['name'].str.contains('Interchange')]
station_list.head()

,name,location.lat,location.lng
0,Wynyard Station,-33.865676,151.206163
4,Town Hall Station,-33.873644,151.206781
6,Artarmon Station,-33.808965,151.183467
8,Ashfield Station,-33.887855,151.125538
9,Petersham Station,-33.893814,151.155502


## 2. Explore venues within 500km from each train stations

### 2.1 Get the venues within 500m from each train station by using API request URL

In [9]:
#create a function to get venues of each train stations
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
#create a new dataframe called station_venues
station_venues = getNearbyVenues(names=station_list['name'],
                                   latitudes=station_list['location.lat'],
                                   longitudes=station_list['location.lng']
                                  )

Wynyard Station
Town Hall Station
Artarmon Station
Ashfield Station
Petersham Station
Mascot Station
St Peters Station
Parramatta Station
St Leonards Station
Redfern Station
Burwood Station
Wolli Creek Station
Hurstville Station
Clyde Station
Sydenham Station
Green Square Station
Strathfield Station
Rhodes Station
International Airport Station
Domestic Airport Station
Newtown Station
Chatswood Station
Milsons Point Station
Edgecliff Station
Campsie Station
Bankstown Station
Kings Cross Station
Auburn Station
Kogarah Station
Gordon Station
Fairfield Station
Stanmore Station
Hornsby Station
Meadowbank Station
Macquarie University Station
Dulwich Hill Station
Central Station
Rockdale Station


In [11]:
# check the size of the resulting dataframe
print(station_venues.shape)
station_venues.head()

(1503, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wynyard Station,-33.865676,151.206163,Regiment,-33.867043,151.206697,Café
1,Wynyard Station,-33.865676,151.206163,Made In Italy,-33.866514,151.205396,Italian Restaurant
2,Wynyard Station,-33.865676,151.206163,Mercado,-33.866901,151.207448,Spanish Restaurant
3,Wynyard Station,-33.865676,151.206163,Nook Urban Fresh Bar,-33.865767,151.204862,Café
4,Wynyard Station,-33.865676,151.206163,City Recital Hall Angel Place,-33.866898,151.207743,Concert Hall


### 2.2 Investigate how many venues were return for each train station

In [12]:
# check how many venues were returned for each train station
station_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Artarmon Station,18,18,18,18,18,18
Ashfield Station,34,34,34,34,34,34
Auburn Station,21,21,21,21,21,21
Bankstown Station,52,52,52,52,52,52
Burwood Station,45,45,45,45,45,45
Campsie Station,20,20,20,20,20,20
Central Station,83,83,83,83,83,83
Chatswood Station,42,42,42,42,42,42
Clyde Station,6,6,6,6,6,6


## 3. Analyze each train station

### 3.1 Create a list of venues for each train station

In [13]:
# one hot encoding
station_onehot = pd.get_dummies(station_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
station_onehot['Neighborhood'] = station_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [station_onehot.columns[-1]] + list(station_onehot.columns[:-1])
station_onehot = station_onehot[fixed_columns]

station_onehot.head()

,Neighborhood,Afghan Restaurant,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Garden,Big Box Store,Bookstore,Boutique,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Station,Café,Candy Store,Cantonese Restaurant,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Duty-free Shop,Egyptian Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food & Drink Shop,Food Court,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hawaiian Restaurant,Health Food Store,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Iraqi Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Motel,Motorcycle Shop,Movie Theater,Multiplex,Music Venue,Newsstand,Nightclub,Noodle House,Office,Other Nightlife,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Soccer Field,South American Restaurant,South Indian Restaurant,Souvlaki Shop,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Wynyard Station,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Wynyard Station,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Wynyard Station,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [14]:
print('There are {} uniques categories.'.format(len(station_venues['Venue Category'].unique())))

There are 196 uniques categories.


### 3.2 Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [15]:
station_grouped = station_onehot.groupby('Neighborhood').mean().reset_index()
station_grouped

,Neighborhood,Afghan Restaurant,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Garden,Big Box Store,Bookstore,Boutique,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Station,Café,Candy Store,Cantonese Restaurant,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Duty-free Shop,Egyptian Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food & Drink Shop,Food Court,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hawaiian Restaurant,Health Food Store,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Iraqi Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Motel,Motorcycle Shop,Movie Theater,Multiplex,Music Venue,Newsstand,Nightclub,Noodle House,Office,Other Nightlife,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Soccer Field,South American Restaurant,South Indian Restaurant,Souvlaki Shop,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Artarmon Station,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.055556,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.222222,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.055556,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.055556,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000

In [16]:
# check data size
station_grouped.shape

(38, 197)

### 3.3 Print each train station along with the top 5 most common venues

In [17]:
num_top_venues = 5

for hood in station_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = station_grouped[station_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Artarmon Station----
                 venue  freq
0                 Café  0.22
1     Asian Restaurant  0.11
2  Japanese Restaurant  0.11
3      Thai Restaurant  0.11
4     Ramen Restaurant  0.06


----Ashfield Station----
                 venue  freq
0  Dumpling Restaurant  0.09
1             Platform  0.06
2  Japanese Restaurant  0.06
3          Supermarket  0.06
4  Shanghai Restaurant  0.06


----Auburn Station----
                venue  freq
0                Café  0.14
1         Supermarket  0.10
2       Grocery Store  0.10
3  Turkish Restaurant  0.10
4   Afghan Restaurant  0.05


----Bankstown Station----
                       venue  freq
0      Vietnamese Restaurant  0.15
1                       Café  0.10
2  Middle Eastern Restaurant  0.06
3              Grocery Store  0.06
4           Department Store  0.04


----Burwood Station----
                  venue  freq
0                  Café  0.11
1    Chinese Restaurant  0.09
2          Noodle House  0.07
3           Supermarket

### 3.4 Put them into a pandas dataframe

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = station_grouped['Neighborhood']

for ind in np.arange(station_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(station_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Artarmon Station,Café,Asian Restaurant,Japanese Restaurant,Thai Restaurant,BBQ Joint,Furniture / Home Store,Sushi Restaurant,Food & Drink Shop,Sandwich Place,Motel,Ramen Restaurant,Convenience Store,Electronics Store,Dive Bar,Fried Chicken Joint,French Restaurant,Fountain,Donut Shop,Dumpling Restaurant,Food Court
1,Ashfield Station,Dumpling Restaurant,Asian Restaurant,Platform,Supermarket,Shanghai Restaurant,Japanese Restaurant,Chinese Restaurant,Electronics Store,Korean Restaurant,Restaurant,Malay Restaurant,Coffee Shop,Liquor Store,Bar,Thai Restaurant,Café,Convenience Store,Bubble Tea Shop,Gym,Train Station
2,Auburn Station,Café,Grocery Store,Turkish Restaurant,Supermarket,Afghan Restaurant,Bakery,Kebab Restaurant,Lebanese Restaurant,Gym,Fast Food Restaurant,Pakistani Restaurant,Persian Restaurant,Dessert Shop,Department Store,Thai Restaurant,Portuguese Restaurant,BBQ Joint,Flea Market,Furniture / Home Store,Fruit & Vegetable Store
3,Bankstown Station,Vietnamese Restaurant,Café,Middle Eastern Restaurant,Grocery Store,Bus Station,Department Store,Buffet,Steakhouse,Chinese Restaurant,Sports Bar,Sports Club,Supermarket,Coffee Shop,Fast Food Restaurant,Discount Store,Portuguese Restaurant,Bubble Tea Shop,Burger Joint,Shopping Mall,Multiplex
4,Burwood Station,Café,Chinese Restaurant,Supermarket,Noodle House,Fast Food Restaurant,Department Store,Japanese Restaurant,Coffee Shop,Sandwich Place,Multiplex,Donut Shop,Fried Chicken Joint,Cantonese Restaurant,Salad Place,Shopping Mall,Bookstore,Big Box Store,Middle Eastern Restaurant,Pub,Juice Bar


## 4. Cluster train stations

### 4.1 Run k-means to cluster the neighborhood into 5 clusters

In [20]:
# set number of clusters
kclusters = 5

station_grouped_clustering = station_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(station_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 2, 1, 1, 1, 2, 2, 4, 2], dtype=int32)

In [21]:
station_list

,name,location.lat,location.lng
0,Wynyard Station,-33.865676,151.206163
4,Town Hall Station,-33.873644,151.206781
6,Artarmon Station,-33.808965,151.183467
8,Ashfield Station,-33.887855,151.125538
9,Petersham Station,-33.893814,151.155502
12,Mascot Station,-33.923162,151.187224
13,St Peters Station,-33.907217,151.180651
14,Parramatta Station,-33.817663,151.005492
15,St Leonards Station,-33.823150,151.194199
16,Redfern Station,-33.891989,151.198750


In [22]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Artarmon Station,Café,Asian Restaurant,Japanese Restaurant,Thai Restaurant,BBQ Joint,Furniture / Home Store,Sushi Restaurant,Food & Drink Shop,Sandwich Place,Motel,Ramen Restaurant,Convenience Store,Electronics Store,Dive Bar,Fried Chicken Joint,French Restaurant,Fountain,Donut Shop,Dumpling Restaurant,Food Court
1,Ashfield Station,Dumpling Restaurant,Asian Restaurant,Platform,Supermarket,Shanghai Restaurant,Japanese Restaurant,Chinese Restaurant,Electronics Store,Korean Restaurant,Restaurant,Malay Restaurant,Coffee Shop,Liquor Store,Bar,Thai Restaurant,Café,Convenience Store,Bubble Tea Shop,Gym,Train Station
2,Auburn Station,Café,Grocery Store,Turkish Restaurant,Supermarket,Afghan Restaurant,Bakery,Kebab Restaurant,Lebanese Restaurant,Gym,Fast Food Restaurant,Pakistani Restaurant,Persian Restaurant,Dessert Shop,Department Store,Thai Restaurant,Portuguese Restaurant,BBQ Joint,Flea Market,Furniture / Home Store,Fruit & Vegetable Store
3,Bankstown Station,Vietnamese Restaurant,Café,Middle Eastern Restaurant,Grocery Store,Bus Station,Department Store,Buffet,Steakhouse,Chinese Restaurant,Sports Bar,Sports Club,Supermarket,Coffee Shop,Fast Food Restaurant,Discount Store,Portuguese Restaurant,Bubble Tea Shop,Burger Joint,Shopping Mall,Multiplex
4,Burwood Station,Café,Chinese Restaurant,Supermarket,Noodle House,Fast Food Restaurant,Department Store,Japanese Restaurant,Coffee Shop,Sandwich Place,Multiplex,Donut Shop,Fried Chicken Joint,Cantonese Restaurant,Salad Place,Shopping Mall,Bookstore,Big Box Store,Middle Eastern Restaurant,Pub,Juice Bar
5,Campsie Station,Malay Restaurant,Korean Restaurant,Department Store,Gym,Platform,Convenience Store,Chinese Restaurant,Fast Food Restaurant,Shopping Mall,Café,Fried Chicken Joint,Frozen Yogurt Shop,Sushi Restaurant,Liquor Store,Playground,Vietnamese Restaurant,Japanese Restaurant,Grocery Store,Electronics Store,Fish Market
6,Central Station,Café,Thai Restaurant,Coffee Shop,Japanese Restaurant,Hostel,Sandwich Place,Bar,Pub,Cocktail Bar,Vietnamese Restaurant,Hotel,Burger Joint,Asian Restaurant,Chinese Restaurant,Speakeasy,Massage Studio,Yoga Studio,Mexican Restaurant,Karaoke Bar,Ramen Restaurant
7,Chatswood Station,Café,Thai Restaurant,Food Court,Coffee Shop,Japanese Restaurant,Gym,Chinese Restaurant,Performing Arts Venue,Chaat Place,Malay Restaurant,Bubble Tea Shop,Ramen Restaurant,Sushi Restaurant,Tea Room,Burger Joint,Portuguese Restaurant,Market,Shopping Mall,Udon Restaurant,Dumpling Restaurant
8,Clyde Station,Platform,Asian Restaurant,Rental Car Location,Train Station,Gym,Falafel Restaurant,Fried Chicken Joint,French Restaurant,Fountain,Food Court,Food & Drink Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Yoga Studio,Event Space,Fruit & Vegetable Store,Electronics Store,Egyptian Restaurant
9,Domestic Airport Station,Café,Coffee Shop,Fast Food Restaurant,Airport Lounge,Juice Bar,Donut Shop,Airport Terminal,Platform,Bar,Sandwich Place,Salad Place,Airport Food Court,Bookstore,Gym,Tapas Restaurant,Airport Gate,Bakery,Electronics Store,History Museum,Asian Restaurant


In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

station_merged = station_list

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
station_merged = station_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='name')

station_merged.head() # check the last columns!

,name,location.lat,location.lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Wynyard Station,-33.865676,151.206163,2,Café,Cocktail Bar,Bar,Speakeasy,Coffee Shop,Sandwich Place,Italian Restaurant,Bakery,Hotel,French Restaurant,Restaurant,Food Court,Steakhouse,Clothing Store,Spanish Restaurant,Dessert Shop,Candy Store,Pub,Chinese Restaurant,Burger Joint
4,Town Hall Station,-33.873644,151.206781,2,Café,Japanese Restaurant,Coffee Shop,Hotel,Cocktail Bar,Bookstore,Korean Restaurant,Burger Joint,Speakeasy,Record Shop,Bar,Tea Room,Thai Restaurant,Australian Restaurant,Ice Cream Shop,Gym,Shopping Mall,Hobby Shop,Hotel Bar,Malay Restaurant
6,Artarmon Station,-33.808965,151.183467,2,Café,Asian Restaurant,Japanese Restaurant,Thai Restaurant,BBQ Joint,Furniture / Home Store,Sushi Restaurant,Food & Drink Shop,Sandwich Place,Motel,Ramen Restaurant,Convenience Store,Electronics Store,Dive Bar,Fried Chicken Joint,French Restaurant,Fountain,Donut Shop,Dumpling Restaurant,Food Court
8,Ashfield Station,-33.887855,151.125538,1,Dumpling Restaurant,Asian Restaurant,Platform,Supermarket,Shanghai Restaurant,Japanese Restaurant,Chinese Restaurant,Electronics Store,Korean Restaurant,Restaurant,Malay Restaurant,Coffee Shop,Liquor Store,Bar,Thai Restaurant,Café,Convenience Store,Bubble Tea Shop,Gym,Train Station
9,Petersham Station,-33.893814,151.155502,0,Café,Portuguese Restaurant,Pub,Pizza Place,Park,Bowling Green,Beer Garden,Chinese Restaurant,Bar,Falafel Restaurant,Grocery Store,Pool,Pharmacy,Electronics Store,Fish Market,Fountain,Food Court,Dive Bar,Food & Drink Shop,Flea Market


### 4.2 Visualize the resulting clusters

In [24]:
# create map
map_clusters = folium.Map(location=[syd_latitude, syd_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(station_merged['location.lat'], station_merged['location.lng'], station_merged['name'], station_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

### 5.1 Cluster 1

In [25]:
station_merged.loc[station_merged['Cluster Labels'] == 0, station_merged.columns[[0] + list(range(4, station_merged.shape[1]))]]

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
9,Petersham Station,Café,Portuguese Restaurant,Pub,Pizza Place,Park,Bowling Green,Beer Garden,Chinese Restaurant,Bar,Falafel Restaurant,Grocery Store,Pool,Pharmacy,Electronics Store,Fish Market,Fountain,Food Court,Dive Bar,Food & Drink Shop,Flea Market
13,St Peters Station,Café,Pub,Theater,Gym / Fitness Center,Thai Restaurant,Italian Restaurant,Grocery Store,Greek Restaurant,Garden Center,Park,Pizza Place,Clothing Store,Indian Restaurant,Athletics & Sports,Australian Restaurant,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
43,Stanmore Station,Café,Park,Pub,Video Store,Beer Garden,Restaurant,Train Station,Farmers Market,French Restaurant,Fountain,Food Court,Food & Drink Shop,Flea Market,Fish Market,Fast Food Restaurant,Event Space,Falafel Restaurant,Frozen Yogurt Shop,Electronics Store,Egyptian Restaurant


### 5.2 Cluster 2

In [26]:
station_merged.loc[station_merged['Cluster Labels'] == 1, station_merged.columns[[0] + list(range(4, station_merged.shape[1]))]]

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
8,Ashfield Station,Dumpling Restaurant,Asian Restaurant,Platform,Supermarket,Shanghai Restaurant,Japanese Restaurant,Chinese Restaurant,Electronics Store,Korean Restaurant,Restaurant,Malay Restaurant,Coffee Shop,Liquor Store,Bar,Thai Restaurant,Café,Convenience Store,Bubble Tea Shop,Gym,Train Station
14,Parramatta Station,Café,Chinese Restaurant,Indian Restaurant,Asian Restaurant,Coffee Shop,Sushi Restaurant,Department Store,Fast Food Restaurant,Thai Restaurant,Sandwich Place,Pub,Vietnamese Restaurant,Supermarket,Juice Bar,Gym / Fitness Center,Burger Joint,Multiplex,Malay Restaurant,Mexican Restaurant,Bakery
17,Burwood Station,Café,Chinese Restaurant,Supermarket,Noodle House,Fast Food Restaurant,Department Store,Japanese Restaurant,Coffee Shop,Sandwich Place,Multiplex,Donut Shop,Fried Chicken Joint,Cantonese Restaurant,Salad Place,Shopping Mall,Bookstore,Big Box Store,Middle Eastern Restaurant,Pub,Juice Bar
19,Hurstville Station,Chinese Restaurant,Fast Food Restaurant,Café,Supermarket,Japanese Restaurant,Coffee Shop,Vietnamese Restaurant,Bakery,Shopping Mall,Middle Eastern Restaurant,Big Box Store,Sandwich Place,Bar,Donut Shop,Juice Bar,Department Store,Dim Sum Restaurant,Dumpling Restaurant,Plaza,Souvlaki Shop
24,Strathfield Station,Korean Restaurant,Café,Vietnamese Restaurant,Bubble Tea Shop,Thai Restaurant,Dessert Shop,Fried Chicken Joint,Sandwich Place,Chinese Restaurant,Bakery,Restaurant,Malay Restaurant,Supermarket,Liquor Store,Bus Station,Fast Food Restaurant,Tennis Court,Sports Bar,Convenience Store,Japanese Restaurant
32,Campsie Station,Malay Restaurant,Korean Restaurant,Department Store,Gym,Platform,Convenience Store,Chinese Restaurant,Fast Food Restaurant,Shopping Mall,Café,Fried Chicken Joint,Frozen Yogurt Shop,Sushi Restaurant,Liquor Store,Playground,Vietnamese Restaurant,Japanese Restaurant,Grocery Store,Electronics Store,Fish Market
33,Bankstown Station,Vietnamese Restaurant,Café,Middle Eastern Restaurant,Grocery Store,Bus Station,Department Store,Buffet,Steakhouse,Chinese Restaurant,Sports Bar,Sports Club,Supermarket,Coffee Shop,Fast Food Restaurant,Discount Store,Portuguese Restaurant,Bubble Tea Shop,Burger Joint,Shopping Mall,Multiplex
42,Fairfield Station,Fast Food Restaurant,Asian Restaurant,Thai Restaurant,Iraqi Restaurant,Pharmacy,Sandwich Place,South American Restaurant,Gym,Thrift / Vintage Store,Gym / Fitness Center,Bowling Alley,Italian Restaurant,Vietnamese Restaurant,Halal Restaurant,Hostel,Food & Drink Shop,Flea Market,Fish Market,Farmers Market,Falafel Restaurant
49,Rockdale Station,Bakery,Coffee Shop,Pastry Shop,Shopping Mall,Grocery Store,Sporting Goods Shop,Sandwich Place,Asian Restaurant,Fast Food Restaurant,Convenience Store,Newsstand,Café,Bus Station,Discount Store,Pizza Place,Food & Drink Shop,Food Court,Flea Market,Yoga Studio,Falafel Restaurant


### 5.3 Cluster 3

In [27]:
station_merged.loc[station_merged['Cluster Labels'] == 2, station_merged.columns[[0] + list(range(4, station_merged.shape[1]))]]

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Wynyard Station,Café,Cocktail Bar,Bar,Speakeasy,Coffee Shop,Sandwich Place,Italian Restaurant,Bakery,Hotel,French Restaurant,Restaurant,Food Court,Steakhouse,Clothing Store,Spanish Restaurant,Dessert Shop,Candy Store,Pub,Chinese Restaurant,Burger Joint
4,Town Hall Station,Café,Japanese Restaurant,Coffee Shop,Hotel,Cocktail Bar,Bookstore,Korean Restaurant,Burger Joint,Speakeasy,Record Shop,Bar,Tea Room,Thai Restaurant,Australian Restaurant,Ice Cream Shop,Gym,Shopping Mall,Hobby Shop,Hotel Bar,Malay Restaurant
6,Artarmon Station,Café,Asian Restaurant,Japanese Restaurant,Thai Restaurant,BBQ Joint,Furniture / Home Store,Sushi Restaurant,Food & Drink Shop,Sandwich Place,Motel,Ramen Restaurant,Convenience Store,Electronics Store,Dive Bar,Fried Chicken Joint,French Restaurant,Fountain,Donut Shop,Dumpling Restaurant,Food Court
12,Mascot Station,Café,Coffee Shop,Hotel,Pizza Place,Thai Restaurant,Australian Restaurant,Breakfast Spot,Chinese Restaurant,Sandwich Place,Sports Bar,Stadium,Clothing Store,Supermarket,Ramen Restaurant,Cosmetics Shop,Athletics & Sports,Halal Restaurant,IT Services,Vietnamese Restaurant,Indian Restaurant
15,St Leonards Station,Café,Coffee Shop,Gym,Vietnamese Restaurant,Chinese Restaurant,Convenience Store,Pub,Souvlaki Shop,Sandwich Place,Restaurant,Playground,Dance Studio,Tennis Court,Pizza Place,Park,Office,Noodle House,Middle Eastern Restaurant,Sushi Restaurant,Hotel
16,Redfern Station,Café,Bar,Pub,Thai Restaurant,Pizza Place,Bakery,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Bus Station,Burger Joint,Cocktail Bar,Restaurant,Tea Room,Dive Bar,Market,Ramen Restaurant,Event Space,Supermarket,Office
22,Sydenham Station,Café,Brewery,Speakeasy,Soccer Field,Coffee Shop,Bar,Liquor Store,Fish Market,Gym,Yoga Studio,Flea Market,Fried Chicken Joint,French Restaurant,Fountain,Food Court,Food & Drink Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
23,Green Square Station,Café,Sporting Goods Shop,Thai Restaurant,Clothing Store,Coffee Shop,Pet Store,Supermarket,Electronics Store,Furniture / Home Store,Train Station,Sushi Restaurant,Gym,Bar,Food Court,Food & Drink Shop,Fountain,Flea Market,French Restaurant,Yoga Studio,Fish Market
25,Rhodes Station,Café,Japanese Restaurant,Fast Food Restaurant,Australian Restaurant,Dim Sum Restaurant,Asian Restaurant,Shopping Mall,Chinese Restaurant,Chocolate Shop,Mexican Restaurant,Food Court,Sandwich Place,Bookstore,Big Box Store,Gym,Movie Theater,American Restaurant,Train Station,Korean Restaurant,Italian Restaurant
26,International Airport Station,Coffee Shop,Airport Lounge,Café,Juice Bar,Electronics Store,Fast Food Restaurant,Hotel,Thai Restaurant,Duty-free Shop,Scenic Lookout,Seafood Restaurant,Mobile Phone Shop,Breakfast Spot,Pie Shop,Lingerie Store,Bakery,Pizza Place,Australian Restaurant,Airport Terminal,Vietnamese Restaurant


### 5.4 Cluster 4

In [28]:
station_merged.loc[station_merged['Cluster Labels'] == 3, station_merged.columns[[0] + list(range(4, station_merged.shape[1]))]]

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
18,Wolli Creek Station,Park,Café,Chinese Restaurant,Bakery,Gym / Fitness Center,Pizza Place,Asian Restaurant,Mediterranean Restaurant,Street Food Gathering,Supermarket,Liquor Store,Grocery Store,Athletics & Sports,Coffee Shop,Train Station,Italian Restaurant,Dumpling Restaurant,French Restaurant,Donut Shop,Fountain
31,Edgecliff Station,Supermarket,Café,Park,Sandwich Place,Train Station,Bakery,Gym,Japanese Restaurant,Italian Restaurant,Food Court,Food & Drink Shop,Flea Market,Fish Market,Fountain,Fast Food Restaurant,French Restaurant,Yoga Studio,Falafel Restaurant,Farmers Market,Frozen Yogurt Shop
41,Gordon Station,Bakery,Thai Restaurant,Coffee Shop,Bus Station,Café,Train Station,Sandwich Place,Park,Fast Food Restaurant,Supermarket,Pizza Place,Food & Drink Shop,Food Court,Fountain,French Restaurant,Flea Market,Yoga Studio,Falafel Restaurant,Fish Market,Farmers Market
45,Meadowbank Station,Grocery Store,Park,Gym,Pharmacy,Thai Restaurant,Shopping Mall,Train Station,Café,Pizza Place,Italian Restaurant,Chinese Restaurant,Fish Market,Falafel Restaurant,Fast Food Restaurant,Farmers Market,Flea Market,Food & Drink Shop,Food Court,Yoga Studio,Event Space


### 5.5 Cluster 5

In [29]:
station_merged.loc[station_merged['Cluster Labels'] == 4, station_merged.columns[[0] + list(range(4, station_merged.shape[1]))]]

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
21,Clyde Station,Platform,Asian Restaurant,Rental Car Location,Train Station,Gym,Falafel Restaurant,Fried Chicken Joint,French Restaurant,Fountain,Food Court,Food & Drink Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Yoga Studio,Event Space,Fruit & Vegetable Store,Electronics Store,Egyptian Restaurant
